In [87]:
%pip install pulp
%pip install scipy
# %pip install pandas
# %pip install numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [88]:
import pandas as pd
import numpy as np
import requests
import io
import datetime
import pulp
import scipy

In [89]:
tickers = ["AEP", "DFSVX", "DFLVX", "FSAGX"]
GS1_URL = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=GS1&scale=left&cosd=1953-04-01&coed=2024-02-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-03-11&revision_date=2024-03-11&nd=1953-04-01"

START_DATE = "01/01/1995"
END_DATE = "09/01/2023"

In [90]:
from yahoo_fin.stock_info import get_data

tickers_data = {}
closed_values = {}
for t in tickers:
    tickers_data[t] = get_data(t, start_date=START_DATE, end_date=END_DATE, index_as_date=False, interval="1mo")
    closed_values[t] = tickers_data[t].iloc[-1]["adjclose"]

print(closed_values)

{'AEP': 75.84803009033203, 'DFSVX': 40.84297180175781, 'DFLVX': 42.006439208984375, 'FSAGX': 21.843658447265625}


In [91]:

gs1_data = requests.get(GS1_URL)
fred = pd.read_csv(io.StringIO(gs1_data.text), parse_dates=["DATE"])
fred = fred.rename(columns={"DATE":"date"})
fred = fred.loc[(fred['date'] >= START_DATE) & (fred['date'] < END_DATE)]

gs1_val = fred.iloc[-1]["GS1"]

'''
for t in tickers:
    df = pd.read_json(f"./data/{t}.json", convert_dates=["Date"])
    merged = pd.merge(merged, df[["date", "Adj Close"]], on="Date", how="inner")
    merged = merged.rename(columns={"Adj Close":f"{t}_Adjusted"})
'''

'\nfor t in tickers:\n    df = pd.read_json(f"./data/{t}.json", convert_dates=["Date"])\n    merged = pd.merge(merged, df[["date", "Adj Close"]], on="Date", how="inner")\n    merged = merged.rename(columns={"Adj Close":f"{t}_Adjusted"})\n'

In [92]:
merged = fred
for t in tickers_data:
    df = tickers_data[t]
    merged = pd.merge(merged, df[["date", "adjclose"]], on="date", how="inner")
    merged = merged.rename(columns={"adjclose":f"{t}_Adjusted"})

In [93]:
shifted = merged.shift(12)
for t in tickers:
    col_to_substract = f"{t}_Adjusted"
    merged[f"{t}_annual"] = (merged[col_to_substract]- shifted[col_to_substract]) / shifted[col_to_substract]
    merged[f"{t}_excess"] = merged[f"{t}_annual"] - merged.shift(12)["GS1"]/100

print(merged.shift(-12).head())

        date   GS1  AEP_Adjusted  DFSVX_Adjusted  DFLVX_Adjusted  \
0 1996-01-01  5.09     12.219304        2.448707        3.688004   
1 1996-02-01  4.94     11.839611        2.502316        3.741763   
2 1996-03-01  5.34     11.688335        2.571488        3.857351   
3 1996-04-01  5.54     11.373382        2.704645        3.935062   
4 1996-05-01  5.64     11.233397        2.810132        3.999793   

   FSAGX_Adjusted  AEP_annual  AEP_excess  DFSVX_annual  DFSVX_excess  \
0       11.230021    0.354033    0.283533      0.275800      0.205300   
1       11.678018    0.355529    0.288529      0.259311      0.192311   
2       11.958015    0.403813    0.339513      0.278872      0.214572   
3       12.242317    0.324276    0.261576      0.299164      0.236464   
4       13.577686    0.250692    0.190692      0.313526      0.253526   

   DFLVX_annual  DFLVX_excess  FSAGX_annual  FSAGX_excess  
0      0.391166      0.320666      0.462142      0.391642  
1      0.335715      0.268715   

In [94]:
return_dist = {}
for t in tickers:
    cleaned = merged[merged[f"{t}_excess"].notna()]
    return_dist[t] = (cleaned[f"{t}_excess"].mean(), cleaned[f"{t}_excess"].std())

print(return_dist)

{'AEP': (0.07754665280376574, 0.19979385241599257), 'DFSVX': (0.10946865468284153, 0.24398202765486768), 'DFLVX': (0.08998956545061457, 0.19464668574460334), 'FSAGX': (0.05298878807301346, 0.30888410177569214)}


In [95]:
## Calulating correlation matrix
cols = [f"{col}_excess" for col in tickers]
rename_dict = {}

for col in cols:
    rename_dict[col] = col.replace("_excess","")
    
corr = merged[cols].corr()
corr = corr.rename(index=rename_dict, columns=rename_dict)
print(corr)

            AEP     DFSVX     DFLVX     FSAGX
AEP    1.000000  0.393708  0.561141 -0.035950
DFSVX  0.393708  1.000000  0.902942  0.091180
DFLVX  0.561141  0.902942  1.000000  0.026616
FSAGX -0.035950  0.091180  0.026616  1.000000


In [96]:
## Calulating variance-covariance
var_covar = corr.copy()
for row_label, row_data in var_covar.iterrows():
    row_std = return_dist[row_label][1]
    for col_label in var_covar.columns:
        col_std = return_dist[col_label][1]
        row_data[col_label] = row_data[col_label]*row_std*col_std

print(var_covar)

            AEP     DFSVX     DFLVX     FSAGX
AEP    0.039918  0.019192  0.021822 -0.002219
DFSVX  0.019192  0.059527  0.042881  0.006872
DFLVX  0.021822  0.042881  0.037887  0.001600
FSAGX -0.002219  0.006872  0.001600  0.095409


In [97]:
weights = np.array(
    [[0.36656],
    [0.32678],
    [0.15174],
    [0.15492]]
    )


print(var_covar.to_numpy())
res = weights.T @ np.power(np.absolute(var_covar.to_numpy()) @ weights,0.5)
sd = res[0,0]

risk_prem = np.dot(weights.T, np.array([[return_dist[r][0]] for r in return_dist.keys()]))[0,0]
sharpie_ratio  = risk_prem/sd
print(risk_prem)


[[ 0.03991758  0.01919173  0.02182234 -0.00221858]
 [ 0.01919173  0.05952723  0.04288097  0.00687155]
 [ 0.02182234  0.04288097  0.03788733  0.00160024]
 [-0.00221858  0.00687155  0.00160024  0.09540939]]
0.08606170773875482


In [99]:
## Here we do the optimization operations.

# Calulcate the min variance portfolio for effecient frontier's SD and riskpremium
import scipy.optimize as optimize
import math
import json 

def calc_risk_premium(wts, return_distr_risk_prem):
    risk_prem = (wts @ return_distr_risk_prem).item()
    return risk_prem

def extend_x(x):
    return [x[0],x[1],x[2],1-x[0]-x[1]-x[2]]

covar = var_covar.to_numpy()
def objective_function(x):
    iter_weights = np.array(extend_x(x))
    return np.power((iter_weights @ covar) @ iter_weights.T ,0.5).item()

def constraint_risk_prem(x, target):
    iter_weights = np.array(extend_x(x))
    return_distr_risk_premium = np.array([[return_dist[r][0]] for r in return_dist.keys()])
    risk_prem = calc_risk_premium(iter_weights, return_distr_risk_premium)
    # print(target, risk_prem)
    return risk_prem - target

def total_one(x):
    return (x[0] + x[1] + x[2]) - 1

# Create the constraint object
total_one_constr = optimize.NonlinearConstraint(total_one, 0, 0)
bounds = [(0,None),(0,None),(0,None)]

weights_map = {}
# Initial guess
initial_values = np.array([0.12, 0.43, 0.23])

# Call the minimize function
result = scipy.optimize.minimize(objective_function, initial_values, method='SLSQP', constraints=[], bounds=bounds)

# Print the result
print("min SD for efficient frontier:", result.fun, result.x)
# print("optimized weights: ", result.x)

risk_prem = np.array(extend_x(result.x)) @ np.array([[return_dist[r][0]] for r in return_dist.keys()])
print("risk prem for efficient frontier 1st line.",risk_prem)

changing_wts = result.x

weights_map[f"{risk_prem.item() * 100}"] = {
    "wts": result.x,
    "sd": result.fun
}

# print(np.array(extend_x(result.x)), np.array([[return_dist[r][0]] for r in return_dist.keys()]), )
next_risk_prem = math.ceil(risk_prem.item() * 100)
for val in range(next_risk_prem, next_risk_prem + 8):
    target_risk_prem = val * 0.01
    # print(target_risk_prem)
    risk_prem_eq_val = optimize.NonlinearConstraint(lambda x: constraint_risk_prem(x,target_risk_prem), 0, 0)
    result = scipy.optimize.minimize(objective_function, changing_wts, method='SLSQP', constraints=[risk_prem_eq_val], bounds=bounds,  tol=1e-6, options={'maxiter': 1000})
    changing_wts = result.x
    # print("min sd:", result.fun, result.x)
   
    weights_map[f"{val}"] = {
        "sd": result.fun,
        "wts": result.x.tolist()
        }

    risk_prem = np.array(extend_x(changing_wts)) @ np.array([[return_dist[r][0]] for r in return_dist.keys()])
    # print(risk_prem[0])

data = {
    "varcovar": var_covar.to_numpy().tolist(),
    "efficient_frontier": weights_map,
    "return_distribution": np.array([[return_dist[r][0]] for r in return_dist.keys()]).tolist(),
    "closed_values": closed_values,
    "gs1_value": gs1_val*0.01,
}



class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# print(json.dumps(data["efficient_frontier"],cls=NumpyEncoder))
    




min SD for efficient frontier: 0.15138863364510582 [0.38243598 0.         0.37501673]
risk prem for efficient frontier 1st line. [0.07625651]


In [103]:
## Getting value for maximum sharpie ratio

import scipy.optimize as optimize
import math
import json 

def calc_risk_premium(wts, return_distr_risk_prem):
    risk_prem = (wts @ return_distr_risk_prem).item()
    return risk_prem

def extend_x(x):
    return [x[0],x[1],x[2],1-x[0]-x[1]-x[2]]

covar = var_covar.to_numpy()
def objective_function(x):
    iter_weights = np.array(extend_x(x))
    return_distr_risk_premium = np.array([[return_dist[r][0]] for r in return_dist.keys()])
    risk_prem = calc_risk_premium(iter_weights, return_distr_risk_premium)
    sd = np.power((iter_weights @ covar) @ iter_weights.T ,0.5).item()
    return sd/risk_prem

# Create the constraint object
bounds = [(0,None),(0,None),(0,None)]
# Initial guess
initial_values = np.array([0.12, 0.43, 0.23])

# Call the minimize function
result = scipy.optimize.minimize(objective_function, initial_values, method='SLSQP', constraints=[], bounds=bounds)

# Print the result
print("max Sharpie ratio:", 1/result.fun, result.x)
# print("optimized weights: ", result.x)

risk_prem = np.array(extend_x(result.x)) @ np.array([[return_dist[r][0]] for r in return_dist.keys()])
print("risk prem for efficient frontier 1st line.",risk_prem)
efficient_sd = result.fun * risk_prem
print("sd.",efficient_sd)

max Sharpie ratio: 0.526918768545048 [0.3758766  0.33213246 0.13739157]
risk prem for efficient frontier 1st line. [0.08606191]
sd. [0.1633305]


In [125]:
## Getting values for complete portfolio

## y, sd, return

complete_portfolio = []
for i in range(0,12):
    sd = i * 0.1 * efficient_sd
        
    if i==0 :
        ex_return = gs1_val * 0.01
    else:
        ex_return = i * 0.1 * risk_prem + gs1_val * 0.01


    complete_portfolio.append({
        "y": i * 0.1,
        "sd": sd.item(),
        "return": ex_return.item(),
    })

print(complete_portfolio)


price_of_risk = risk_prem/efficient_sd**2
risk_aver = 4 # this is also an arbitraty value.
y = price_of_risk/risk_aver

DGS1 = gs1_data*0.01     # This is the cuurent GS1 value for the current month.
E_rf = (1-y)*DGS1
E_rP = y * (DGS1+risk_prem)

expected_return =  E_rf+E_rP
riskiness = y * efficient_sd

[{'y': 0.0, 'sd': 0.0, 'return': 0.053700000000000005}, {'y': 0.1, 'sd': 0.01633305013745721, 'return': 0.062306190665013486}, {'y': 0.2, 'sd': 0.03266610027491442, 'return': 0.07091238133002696}, {'y': 0.30000000000000004, 'sd': 0.048999150412371635, 'return': 0.07951857199504045}, {'y': 0.4, 'sd': 0.06533220054982884, 'return': 0.08812476266005392}, {'y': 0.5, 'sd': 0.08166525068728604, 'return': 0.0967309533250674}, {'y': 0.6000000000000001, 'sd': 0.09799830082474327, 'return': 0.1053371439900809}, {'y': 0.7000000000000001, 'sd': 0.11433135096220047, 'return': 0.11394333465509437}, {'y': 0.8, 'sd': 0.13066440109965768, 'return': 0.12254952532010785}, {'y': 0.9, 'sd': 0.14699745123711488, 'return': 0.13115571598512132}, {'y': 1.0, 'sd': 0.16333050137457208, 'return': 0.1397619066501348}, {'y': 1.1, 'sd': 0.1796635515120293, 'return': 0.1483680973151483}]


In [101]:
# import scipy.optimize


# covar = var_covar.to_numpy()

# def calc_risk_premium(wts, return_distr_risk_prem):
#     risk_prem = wts @ return_distr_risk_prem
#     return risk_prem

# def objective_function(x):
#     iter_weights = np.array(x)
#     return np.power((iter_weights @ covar) @ iter_weights.T ,0.5).item() - total_one(x)

# def constraint_risk_prem(x, target):
#     iter_weights = np.array(x)
#     return_distr_risk_premium = np.array([[return_dist[r][0]] for r in return_dist.keys()])
#     risk_prem = calc_risk_premium(iter_weights, return_distr_risk_premium)
#     return risk_prem - target

# def total_one(x):
#     return (x[0] + x[1] + x[2] + x[3]) - 1

# # Define the bounds for each variable
# bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

# # Initial guess
# initial_values = np.array([0.1, 0.2, 0.3, 0.4])

# # Define the constraint function for fmin_l_bfgs_b
# def constraint_function(x):
#     # Assuming a target risk premium
#     target_risk_premium = 0.05 # Example value
#     risk_prem_diff = constraint_risk_prem(x, target_risk_premium)
#     total_one_diff = total_one(x)
#     return (risk_prem_diff, total_one_diff)

# # Call the fmin_l_bfgs_b function
# result = scipy.optimize.fmin_l_bfgs_b(objective_function, initial_values, bounds=bounds, approx_grad=True)

# print(result)